In [1]:
import os

folder_path = '/Users/kimseonjin/Desktop/jet'

file_list = os.listdir(folder_path)

print(f'폴더 {folder_path} 내에는 {len(file_list)}개의 파일이 있습니다.')

폴더 /Users/kimseonjin/Desktop/jet 내에는 24974개의 파일이 있습니다.


환경설정

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
from tqdm import tqdm
import random
from shutil import move


In [14]:
csv_paths= glob.glob('/Users/kimseonjin/Desktop/jet/*.csv')
jpg_paths= glob.glob('/Users/kimseonjin/Desktop/jet/*.jpg')
csv_paths.sort()
jpg_paths.sort()

dataset에 대한 정보 파악

In [4]:

csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
csv_paths = [os.path.join(folder_path, file) for file in csv_files]

dfs = [pd.read_csv(csv_path) for csv_path in csv_paths]
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

print(combined_df)

                               filename  width  height   class  xmin  ymin  \
0      5dc92840a1c2570658f210118ee712af   4096    2730     F15   253  1070   
1      5dc92840a1c2570658f210118ee712af   4096    2730     F15  1680   908   
2      ce0ef7f5181357f27e09a98f4f07aa26   1600    1075    SR71   216   345   
3      769c27a7b2c804a73458175fd13a3a53    916     316     J20   303   183   
4      ac6c99e73c2b4a6af7a9d46b7209bd32   6016    4016      F4     1  1759   
...                                 ...    ...     ...     ...   ...   ...   
20153  313d6f38ab74d109dddb5efd382aeacb   1000     563      C2   453   210   
20154  059843e8192d9e4e990da370775e1b53   1600     900     F18   213   200   
20155  9cf79ef94af7f2c32882b79b14493761   3520    1813  EF2000  2115   250   
20156  9cf79ef94af7f2c32882b79b14493761   3520    1813  EF2000  1296   602   
20157  6d761316b0a4b33dab3ebcf386f5f184   7360    4912      E2  2448  1605   

       xmax  ymax  
0      2275  1627  
1      4057  1656  
2  

In [5]:
print(combined_df['class'].value_counts())


F18           1234
F16           1191
F15           1158
F35           1033
C130           853
V22            751
C2             750
US2            581
A10            576
F22            569
B1             548
F4             536
EF2000         514
C17            477
Rafale         472
J20            461
B52            461
F14            458
JAS39          445
B2             436
Tornado        390
E2             389
A400M          385
Mirage2000     368
AV8B           362
Vulcan         343
C5             340
Mig31          325
Su34           322
F117           316
MQ9            309
Su57           299
RQ4            280
Tu160          258
U2             257
Be200          255
SR71           248
Tu95           247
P3             206
AG600          194
XB70           164
YF23           131
E7             130
J10             70
Su25            38
KC135           28
Name: class, dtype: int64


In [6]:
len(combined_df['class'].unique())

46

class간 데이터의 불균형이 매우 심하다-> partition problem을 일으킬 수 있다. 편의상 제거하고 분석하도록 하겠다. 어차피 이미지 데이터는 꽤 많다. 다음은 Su57 밑에 있는 데이터들을 모두 제거하는 스텝이다.

In [7]:
target_classes = ['Su57', 'RQ4', 'Tu160', 'U2', 'Be200', 'Tu95', 'SR71', 'P3', 'AG600', 'XB70', 'YF23', 'E7']

# 이동할 디렉토리 경로
destination_path = '/Users/kimseonjin/Desktop/delete'

# 디렉토리 내 모든 CSV 파일 경로 가져오기
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
csv_paths = [os.path.join(folder_path, file) for file in csv_files]

# 주어진 클래스에 해당하는 파일을 이동
for csv_path in csv_paths:
    df = pd.read_csv(csv_path)
    if df['class'].isin(target_classes).any():
        move(csv_path, os.path.join(destination_path, os.path.basename(csv_path)))
        print(f"Moved: {csv_path} to {destination_path}")

Moved: /Users/kimseonjin/Desktop/jet/ce0ef7f5181357f27e09a98f4f07aa26.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/db1487f376b937f7a30737ce4d995a15.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/4a6b0a99c9e8c0b93ec1b6d646573b60.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/1449ed1726c3f56fffe518244afcc828.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/5ac91f8fe70885f3f56662c42c3ea190.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/122d7ee3c1a2931db985f9249a41d004.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/8604a8374312c60089f19172617da70a.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/dba44cb17caceba2e0bf372be7b1fa48.csv to /Users/kimseonjin/Desktop/delete
Moved: /Users/kimseonjin/Desktop/jet/b4e8716260cf7713f0abe07714520fb4.csv to /Users/kimseonjin/Desktop/delete
Moved: /Us

In [8]:
csv_directory = '/Users/kimseonjin/Desktop/delete'
jpg_directory = '/Users/kimseonjin/Desktop/jet'

csv_files = [file.split('.')[0] for file in os.listdir(csv_directory) if file.endswith('.csv')]
jpg_files = [file for file in os.listdir(jpg_directory) if file.endswith('.jpg')]

for jpg_file in jpg_files:
    jpg_file_name = os.path.splitext(jpg_file)[0]
    if jpg_file_name in csv_files:
        source_path = os.path.join(jpg_directory, jpg_file)
        destination_path = os.path.join(csv_directory, jpg_file)
        move(source_path, destination_path)
        print(f"Moved: {source_path} to {destination_path}")

Moved: /Users/kimseonjin/Desktop/jet/1624eca3a678e90084c83ff10f516045.jpg to /Users/kimseonjin/Desktop/delete/1624eca3a678e90084c83ff10f516045.jpg
Moved: /Users/kimseonjin/Desktop/jet/d7c0e3bf4acad7703539d7041c6dedba.jpg to /Users/kimseonjin/Desktop/delete/d7c0e3bf4acad7703539d7041c6dedba.jpg
Moved: /Users/kimseonjin/Desktop/jet/b468cf3161d72099b021b35c3b079000.jpg to /Users/kimseonjin/Desktop/delete/b468cf3161d72099b021b35c3b079000.jpg
Moved: /Users/kimseonjin/Desktop/jet/f007598dacc4b6baa1aae36229c4d594.jpg to /Users/kimseonjin/Desktop/delete/f007598dacc4b6baa1aae36229c4d594.jpg
Moved: /Users/kimseonjin/Desktop/jet/25026b38c8fdd1fdf60e199d573121e9.jpg to /Users/kimseonjin/Desktop/delete/25026b38c8fdd1fdf60e199d573121e9.jpg
Moved: /Users/kimseonjin/Desktop/jet/218c63a757cb31e3acd696ffe08609b4.jpg to /Users/kimseonjin/Desktop/delete/218c63a757cb31e3acd696ffe08609b4.jpg
Moved: /Users/kimseonjin/Desktop/jet/08b11baa46e2d9185dd9ee4af5ce89fc.jpg to /Users/kimseonjin/Desktop/delete/08b11baa

In [9]:
classes= np.array(['F18', 'F16', 'F15', 'F35', 'C130', 'V22' ,'C2' ,'US2', 'A10', 'F22', 'B1', 'F4', 'EF2000', 'C17', 'Rafale', 'J20', 'B52', 'F14', 'JAS39', 'B2', 'Tornado', 'E2', '389A400M','Mirage2000','AV8B' ,'Vulcan','C5','Mig31','Su34','F117','MQ9'])

train valid test split을 위한 directory 형성

In [10]:
os.makedirs('/Users/kimseonjin/Desktop/jet_detection/train/images', exist_ok= True)
os.makedirs('/Users/kimseonjin/Desktop/jet_detection/train/labels', exist_ok= True)

os.makedirs('/Users/kimseonjin/Desktop/jet_detection/valid/images', exist_ok= True)
os.makedirs('/Users/kimseonjin/Desktop/jet_detection/valid/labels', exist_ok= True)

os.makedirs('/Users/kimseonjin/Desktop/jet_detection/test/images', exist_ok= True)
os.makedirs('/Users/kimseonjin/Desktop/jet_detection/test/labels', exist_ok= True)

In [11]:
train_folder= '/Users/kimseonjin/Desktop/jet_detection/train'
valid_folder= '/Users/kimseonjin/Desktop/jet_detection/valid'
test_folder= '/Users/kimseonjin/Desktop/jet_detection/test'

train valid test split

In [15]:
train_ratio= 0.7
valid_ratio= 0.15
test_ratio= 0.15

random.seed(42)
data_pairs= list(zip(csv_paths, jpg_paths))
random.shuffle(data_pairs)

total_files= len(data_pairs)
train_end= int(total_files*train_ratio)
valid_end= int(total_files*(train_ratio+valid_ratio))

In [20]:
for i, (csv_paths, jpg_paths) in enumerate(tqdm(data_pairs)):
    
    annotations = np.array(pd.read_csv(csv_paths))

    
    if i < train_end:
        jpg_file_path = '/Users/kimseonjin/Desktop/jet_detection/train/images/' + os.path.basename(jpg_paths)
        txt_file_path = '/Users/kimseonjin/Desktop/jet_detection/train/labels/' + os.path.basename(csv_paths)[:-4] + '.txt'
        shutil.copy(jpg_paths, jpg_file_path)
    elif i < valid_end:
        jpg_file_path = '/Users/kimseonjin/Desktop/jet_detection/valid/images/' + os.path.basename(jpg_paths)
        txt_file_path = '/Users/kimseonjin/Desktop/jet_detection/valid/labels/' + os.path.basename(csv_paths)[:-4] + '.txt'
        shutil.copy(jpg_paths, jpg_file_path)
    else:
        jpg_file_path = '/Users/kimseonjin/Desktop/jet_detection/test/images/' + os.path.basename(jpg_paths)
        txt_file_path = '/Users/kimseonjin/Desktop/jet_detection/test/labels/' + os.path.basename(csv_paths)[:-4] + '.txt'
        shutil.copy(jpg_paths, jpg_file_path)

    with open(txt_file_path, mode='w') as f:
        try:
            for annotation in annotations:
                width = annotation[1]
                height = annotation[2]
                class_name = annotation[3]
                xmin = annotation[4]
                ymin = annotation[5]
                xmax = annotation[6]
                ymax = annotation[7]
                x_center = 0.5*(xmin+xmax)
                y_center = 0.5*(ymin+ymax)
                b_width = xmax - xmin
                b_height= ymax - ymin
                class_num = np.where(classes==class_name)[0][0]
                output_string = '{} {} {} {} {}\n'.format(class_num,
                                                        x_center/width,
                                                        y_center/height,
                                                        b_width/width,
                                                        b_height/height)
                f.write(output_string)
        except:
            print(txt_file_path)

  0%|          | 0/10338 [00:00<?, ?it/s]

  0%|          | 38/10338 [00:00<00:55, 187.06it/s]

/Users/kimseonjin/Desktop/jet_detection/train/labels/9aa174ca51106600066f5a1911682599.txt


  1%|          | 85/10338 [00:00<00:46, 222.40it/s]

/Users/kimseonjin/Desktop/jet_detection/train/labels/ad69f0a98d7846c48838902e6f8286c9.txt
/Users/kimseonjin/Desktop/jet_detection/train/labels/6305aa0be60d4aada2b56a12ed74c593.txt
/Users/kimseonjin/Desktop/jet_detection/train/labels/abaea0294909c4357a9723026090a372.txt


...

 99%|█████████▉| 10219/10338 [00:49<00:00, 204.72it/s]

/Users/kimseonjin/Desktop/jet_detection/test/labels/f45b550851aedd82b46eb3c682ebb993.txt
/Users/kimseonjin/Desktop/jet_detection/test/labels/677e0eb8e32a5074bfd8130e9679f92e.txt
/Users/kimseonjin/Desktop/jet_detection/test/labels/f32aa69bd9b10f9fc9b1173b92be7080.txt


100%|█████████▉| 10313/10338 [00:49<00:00, 208.30it/s]

/Users/kimseonjin/Desktop/jet_detection/test/labels/9ba69ee94e628ebfc7369b521b4ab145.txt
/Users/kimseonjin/Desktop/jet_detection/test/labels/9b8fb531f78a285eb90df67764f0e868.txt


100%|██████████| 10338/10338 [00:49<00:00, 207.44it/s]


In [21]:
classes_string = ', '.join([f'"{c}"' for c in classes])
with open('/Users/kimseonjin/Desktop/jet_detection/mad.yaml', mode='w') as f:
    yaml_string='train: /content/drive/MyDrive/jet_detection/train\n'\
              + 'val: /content/drive/MyDrive/jet_detection/valid\n'\
              + 'test: /content/drive/MyDrive/jet_detection/test\n'\
              + f'nc: {len(classes)}\n'\
              + f'names: [{classes_string}]'
    f.write(yaml_string)
print(yaml_string)

train: /content/drive/MyDrive/jet_detection/train
val: /content/drive/MyDrive/jet_detection/valid
test: /content/drive/MyDrive/jet_detection/test
nc: 31
names: ["F18", "F16", "F15", "F35", "C130", "V22", "C2", "US2", "A10", "F22", "B1", "F4", "EF2000", "C17", "Rafale", "J20", "B52", "F14", "JAS39", "B2", "Tornado", "E2", "389A400M", "Mirage2000", "AV8B", "Vulcan", "C5", "Mig31", "Su34", "F117", "MQ9"]
